In [1]:
%cd ..
import os, sys
sys.path.append(os.path.join(os.getcwd(), "src"))

d:\Blockchain\data\orai\orchai_validator


In [2]:
from orchai.upload import get_spark, sampling, uploading, run_uploading
from orchai.constants import Constants
from orchai.etl_processor import validating, ETLProcessor
from omegaconf import OmegaConf
import pyspark.sql.functions as F
from pyspark.sql import DataFrame, Window
import numpy as np

In [3]:
spark = get_spark()
config = OmegaConf.load("config/etl_file_1m.yaml")
sb, eb = 7059473, 9583823
print((eb - sb) / 150 + 1)

16830.0


In [4]:
# run_uploading(config, sb, eb, spark, cal_score=True, overwrite=True)

------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
start-end final blocks: 8920523 9506273


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "d:\Apps\MiniConda3\envs\py38\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "d:\Apps\MiniConda3\envs\py38\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "d:\Apps\MiniConda3\envs\py38\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\Apps\MiniConda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_19632\2188806573.py", line 1, in <module>
    dt = ETLProcessor.data_scoring(sampling(spark, config, from_block=8906273, to_block=9506273), cal_score=True, **config.hp.etl)
  File "d:\Blockchain\data\orai\orchai_validator\src\orchai\etl_processor.py", line 100, in data_scoring
    df = ETLProcessor.shifting_data(df, label_win_size)
  File "d:\Blockchain\data\orai\orchai_validator\src\orchai\etl_processor.py", line 272, in shifting_data
    }
  File "d:\Blockchain\data\orai\orchai_validator\src\orchai\etl_processor.py", line 273, in <dictcomp>
  File "d:\Apps\MiniConda3\envs\py38\lib\site-packages\pyspark\sql\dataframe.py", line 804, in count
    return int(self._jdf.count())
  File "d:\Apps\MiniConda3\envs\py38\lib\site-packages\py4j\jav